In [76]:
import gym
import sys
import pylab
import random
import os
import operator
from collections import deque

from skimage import io, color, transform

import numpy as np

'''Pick your poison: Keras, Pytorch, Tensorflow'''
from keras.layers import Conv2D, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import Adam

GAME_TYPE = 'MsPacman-v0'
env =gym.make("MsPacman-v0")   # USE this env: MsPacman-v0
# import your desired environment, might need to download some dependancies for atari environments. Prepend ! to use cli commands

In [77]:
#####
# Hyperparameters
#####

''' These are not the golden hyperparameters, just put there to give a rough idea of magnitude. Search for parameters in this ball park'''
#environment parameters
NUM_EPISODES = 80000000
PHI_LENGTH = 4

#agent parameters
EPSILON = 1
EXPERIENCE_REPLAY_CAPACITY = 2000
MINIBATCH_SIZE = 100
LEARNING_RATE = 0.001
ACTION_SIZE = env.action_space.n
EXPLORE = 3000000
PREPROCESS_IMAGE_DIM = 84

print(ACTION_SIZE)

9


# Playground

In [78]:
class Agent:
    def __init__(self):
        self.processed_image_dim = PREPROCESS_IMAGE_DIM 
        
    def preprocess_observation(self, observation, prediction=False):
        """
        Helper function for preprocessing an observation for consumption by our
        deep learning network
        """
        grayscale_observation = color.rgb2gray(observation)
        resized_observation = transform.resize(grayscale_observation, (1, self.processed_image_dim, self.processed_image_dim)).astype('float32')
        if prediction:
            resized_observation = np.expand_dims(resized_observation, 0)
        return resized_observation
      
obj = Agent() 

In [79]:
'''
Use this cell just to explore the environment, not used towards the algo
'''
state =env.reset()
#print state
print("State.shape = %s" % (state.shape,))
print ("State_size = %s" % (state.size,))
#laying out all features in 1 row, for fun
state = np.reshape(state, [1, state.size])
#print state
print ("State.shape now = %s" % (state.shape,))
print ("State_size now = %s" % (state.size,))
#print env.observation_space
print ("env.observation_space.shape = %s" %(env.observation_space.shape,))

action = random.randrange(env.action_space.n)
next_state, reward, done, info = env.step(action)
#print next_state
print ("next_State.shape = %s" % (next_state.shape,))
next_state = np.reshape(state, [1, next_state.size])
#print next_state
print ("next_State.shape now = %s" % (next_state.shape,))

#slight change of notations
#ok let next_state be called x_t1 and current state be x_t
# let reward be r_t and action a_t, 9=env.action_space.n= possible actions


x_t = env.reset()
x_t = obj.preprocess_observation(x_t)
    #print ("x_t.shape = %s"% (x_t.shape,))


s_t = np.stack((x_t, x_t, x_t, x_t), axis=3)
print ("s_t.shape = %s"% (s_t.shape,))

    #above stuff - one time per episode at the start eventualy set s_t = s_t1 and so on

a_t = random.randrange(env.action_space.n)
x_t1 , r_t, done, info = env.step(a_t)
print (x_t1.shape)
x_t1= obj.preprocess_observation(x_t1)
print ("x_t1.shape = %s"% (x_t1.shape,))
x_t1 = x_t1.reshape(x_t1.shape[0], x_t1.shape[1], x_t1.shape[2],1) #1x84x84x1
print ("x_t1.shape now = %s"% (x_t1.shape,))
s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3)
print ("s_t1.shape = %s"% (s_t1.shape,))
        #s_t = s_t1
    #now s_t =s_t1 and then loop

State.shape = (210, 160, 3)
State_size = 100800
State.shape now = (1, 100800)
State_size now = 100800
env.observation_space.shape = (210, 160, 3)
next_State.shape = (210, 160, 3)
next_State.shape now = (1, 100800)
s_t.shape = (1, 84, 84, 4)
(210, 160, 3)
x_t1.shape = (1, 84, 84)
x_t1.shape now = (1, 84, 84, 1)
s_t1.shape = (1, 84, 84, 4)


In [80]:
#visualising deque
D = deque(maxlen=5)
D.append((x_t, a_t, r_t, x_t1, done))
D.append((s_t, a_t, r_t, s_t1, done))
print(D)

deque([(array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32), 5, 0.0, array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]]], dtype=float32), False), (array([[[[0., 0., 0., 0.],
         [0., 0.

#Main Program

In [98]:
#lets begin
UPDATE_EVERY = 5
class Agent:

    def __init__(self, epsilon , experience_replay_capacity , minibatch_size , learning_rate ,action_size, img_dim , explore):
        
        self.action_size = action_size
        self.discount_factor = 0.99
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.epsilon_min = 0.01
        self.batch_size = minibatch_size
        self.train_start = 1000
        self.explore = explore
        self.img_channels = 4 #phi_length  #coz we feed in 4 stacked b&w imgs instead of 1 rbg img
        self.processed_image_dim = img_dim
        
         # create replay memory using deque
        self.D = deque(maxlen=experience_replay_capacity)
        # create main model and target model
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.target_update_counter = 0

        # initialize target model
        self.target_model.set_weights(self.model.get_weights())
        
    def build_model(self) :
        '''Write your deep learning model here eg. CNN or RNN etc'''
        model = Sequential()
        model.add(Conv2D(32, (8,8), padding='same', strides=(4,4), input_shape=(84, 84, 4)))
        model.add(Conv2D(64, (4,4), padding='same', strides=(2,2)))
        model.add(Conv2D(64, (3,3), padding='same', strides=(1,1)))
        
        model.add(Flatten())
        model.add(Dense(128))
        model.add(Dense(ACTION_SIZE, activation='linear'))
        
        model.compile(loss='categorical_crossentropy', optimizer=Adam(lr = LEARNING_RATE), metrics=['accuracy'])
        #try printing model summary
        print(model.summary())
        #FILL THIS
        print("finish building the model")
        return model
      
    # after some time interval update the target model to be same with model ###Q-learning specific function
    '''def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())
        #FILL '''
        
    def append_experience_replay_example(self,s_t,a_t,r_t,s_t1,done):
        """
        Add an experience replay example to our agent's replay memory. If
        memory is full, overwrite previous examples, starting with the oldest
        """
        D.append((s_t, a_t, r_t, s_t1, done))
       
        if self.epsilon > self.epsilon_min :
            self.epsilon -= (self.epsilon - self.epsilon_min) /self.explore
            #this is just one way of reducing exploration, try out other ways, experiment
        
    def preprocess_observation(self, observation, prediction=False):
        """
        Helper function for preprocessing an observation for consumption by our
        deep learning network
        """
        grayscale_observation = color.rgb2gray(observation)
        resized_observation = transform.resize(grayscale_observation, (1, self.processed_image_dim, self.processed_image_dim)).astype('float32')
        if prediction:
            resized_observation = np.expand_dims(resized_observation, 0)
        return resized_observation
        
    def take_action(self, s_t):
        """
        Given an observation, the model attempts to take an action
        according to its q-function approximation
        """
        k = self.model.predict(np.array(s_t).reshape(-1, *s_t.shape))[0]
        action = np.argmax(k)
        return action
        #FILL THIS
        
    def learn(self, terminal_state, step):
        """
        Allow the model to collect examples from its experience replay memory
        and learn from them
        """
        #this is similar to your train loop or model.fit for those not comfortable with RL
        #FILL THIS
        if len(self.D)<MIN_REPLAY_CAPACITY:
            return
        
        minibatch = random.sample(self.D, minibatch_size)
        
        #get the q vals
        current_states = np.array([s_t for s_t in minibatch_size])
        current_qs_list = self.model.predict(current_states)
        
        new_current_states = np.array([s_t1 for s_t1 in minibatch_size])
        future_qs_list = self.target_model.predict(new_current_states)
        
        X = []
        y = []
        
        for index, (s_t, a_t, r_t, s_t1, done) in enumerate(minibatch_size):
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = r_t + discount_factor * max_future_q
            else:
                new_q = r_t
                
            current_qs = current_qs_list[index]
            current_qs[action] = new_qs
            
            X.append(s_t)
            y.append(cuurent_qs)
            
        self.model.fit(np.array(X), np.array(y), verbose=0, batch_size=minibatch_size, shuffle=False)
        if terminal_state:
            self.target_update_counter+=1
        
        if self.target.update_counter>UPDATE_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

In [101]:
MIN_REPLAY_CAPACITY = 5
step = 0
def run_simulation():
    """
    Entry-point for running env simulation
    """

    #print game parameters
    print ("~~~Environment Parameters~~~")
    print ("Num episodes: %s" % NUM_EPISODES)
    print ("Action space: %s" % env.action_space)
    print()
    print ("~~~Agent Parameters~~~")
    print ("Epsilon: %s" % EPSILON)
    print ("Experience Replay Capacity: %s" % EXPERIENCE_REPLAY_CAPACITY)
    print ("Minibatch Size: %s" % MINIBATCH_SIZE)
    print ("Learning Rate: %s" % LEARNING_RATE)

    #initialize agent
    agent = Agent(epsilon=EPSILON,
                experience_replay_capacity=EXPERIENCE_REPLAY_CAPACITY,
                minibatch_size=MINIBATCH_SIZE,
                learning_rate=LEARNING_RATE, action_size =ACTION_SIZE, img_dim =PREPROCESS_IMAGE_DIM ,explore =EXPLORE)
    
    scores, episodes = [], [] 

    #initialize auxiliary data structures
    state_list = [] 
    tot_frames = 0

    for i_episode in range(NUM_EPISODES):
        print ("Episode: %s" % i_episode)
         
        done = False
        score = 0
        step = 0
        x_t = env.reset()
        x_t = agent.preprocess_observation(x_t)   
        s_t = np.stack((x_t, x_t, x_t, x_t), axis=3) #how many consecutive frames to stack depends on your PHI
         

        
        
        while not done:
            #env.render()
           # get action for the current state and go one step in environment
           # get action, change score and learn from memory
           #FILL THIS
            if np.random.random()>EPSILON:
                a_t = agent.take_action(s_t)
            else:
                a_t = env.action_space.sample()
                
            s_t1, r_t, done, _ = env.step(action)
            score += r_t
            agent.append_experience_replay_example(s_t, a_t, r_t, s_t1, done)
            agent.learn(done, step)
            step+=1
            s_t = s_t1
            
        if done:
            # every episode update the target model to be same with model
            #agent.update_target_model() 
            scores.append(score)
            episodes.append(i_episode)
            print( "  score:", score, "  epsilon:", agent.epsilon)
            
        while True:
           
            #ensure state list is populated
            if tot_frames < PHI_LENGTH:
                state_list.append(agent.preprocess_observation(x_t))
                tot_frames += 1
                continue



                #update state list with next observation
                state_list.append(preprocess_observation(x_t))
                state_list.pop(0)


In [102]:
if __name__ == "__main__":
    env =gym.make("MsPacman-v0") #env name
    run_simulation()

~~~Environment Parameters~~~
Num episodes: 80000000
Action space: Discrete(9)

~~~Agent Parameters~~~
Epsilon: 1
Experience Replay Capacity: 2000
Minibatch Size: 100
Learning Rate: 0.001
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 21, 21, 32)        8224      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 11, 11, 64)        32832     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 64)        36928     
_________________________________________________________________
flatten_29 (Flatten)         (None, 7744)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               991360    
_________________________________________________________________
dense_9 (Dense)      

KeyboardInterrupt: 